### 학습목표
- bmi 데이터를 사용하여 머신러닝의 전체과정을 이해해보자
- knn 분류 모델을 사용해보자
- 하이퍼파라미터를 조정해보자

### 머신러닝 종류
1. 지도학습
   - 문제와 답이 주어진 상태에서 학습
   - 분류, 회귀
       - 분류 : 정답 데이터의 형태가 범주형 데이터일 때
       - 회귀 : 정답 데이터의 형태가 연속형 데이터일 때

2. 비지도학습
    - 문제에 대해서만 주어신 상태에서 학습(답을 알려주지 않는다)
    - 데이터의 숨겨진 특성, 구조, 패턴을 파악 -> 클러스터링(군집화)

3. 강화학습
    - 완전한 답을 제공하지 않음
    - 더 큰 보상을 얻을 수 있는 방향으로 학습을 진행
    - ex) 알파고

### 머신러닝 단계
1. 문제 정의 : 지도 vs 비지도, 필요한 데이터 정의, 분류 vs 회귀
2. 데이터 수집 : 크롤링, db, 공공데이터포털, 설문조사
3. 데이터 전처리 : numpy, pandas 활용
4. 탐색적 데이터 분석(EDA)
    - 특성 의미 파악(기술통계 : 평균, 최소, 최대, 중앙값, 최빈, 표준편차...)
    - 특성간의 상관관계
    - 그래프를 출력해서 의미 파악 : numpy, pandas, matplotlib
5. 모델 선택, 하이퍼파라미너(사람 지정 매개변수) 조정
6. 모델 학습
7. 모델 평가 및 예측

### 1. 문제 정의
- 500명의 키, 몸무게 데이터를 통해서 비만도를 판별하는 모델 생성
- 분류 모델활용 -> 6가지의 class로 분류하는 판별기를 만들어보자!

In [2]:
# 필요한 도구 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 2. 데이터 수집
- csv 파일 불러오기

In [3]:
data = pd.read_csv('./data/bmi_500.csv')
data

,Gender,Height,Weight,Label
0,Male,174,96,Obesity
1,Male,189,87,Normal
2,Female,185,110,Obesity
3,Female,195,104,Overweight
4,Male,149,61,Overweight
...,...,...,...,...
495,Female,150,153,Extreme Obesity
496,Female,184,121,Obesity
497,Female,141,136,Extreme Obesity
498,Male,150,95,Extreme Obesity


### 3. 데이터 전처리
- 결측치 대체 or 제거
- 이상치 대체 or 제거
- 데이터 split(train set과 test set을 분리)

In [4]:
# 위에서부터 5개 행만 출력
data.head()

,Gender,Height,Weight,Label
0,Male,174,96,Obesity
1,Male,189,87,Normal
2,Female,185,110,Obesity
3,Female,195,104,Overweight
4,Male,149,61,Overweight


In [5]:
# 아래부터 5개 행만 출력
data.tail()

,Gender,Height,Weight,Label
495,Female,150,153,Extreme Obesity
496,Female,184,121,Obesity
497,Female,141,136,Extreme Obesity
498,Male,150,95,Extreme Obesity
499,Male,173,131,Extreme Obesity


In [6]:
# 데이터 크기 정보 확인
data.shape

(500, 4)

In [7]:
# 데이터 차원 확인
data.ndim

2

In [8]:
# 데이터 정보 확인
# 컬럼명, 컬럼개수, 행 개수, 결측치여부, 데이터 타입
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gender  500 non-null    object
 1   Height  500 non-null    int64 
 2   Weight  500 non-null    int64 
 3   Label   500 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


### 4. EDA(탐색적 데이터 분석)
- 데이터의 특성 확인, 파악
- 기술통계량(요약본), 데이터의 분포 파악, 시각화

In [11]:
# 기술통계량(요약본) 확인
# df.describe()
# 수치명 테이터에 대해 요약본 확인
data.describe()

,Height,Weight
count,500.000000,500.000000
mean,169.944000,106.000000
std,16.375261,32.382607
min,140.000000,50.000000
25%,156.000000,80.000000
50%,170.500000,106.000000
75%,184.000000,136.000000
max,199.000000,160.000000


In [12]:
# object형에 대해 기술통계량 확인
data.describe(include = 'object')

,Gender,Label
count,500,500
unique,2,6
top,Female,Extreme Obesity
freq,255,198


In [17]:
# 정답 데이터 확인하기(label)
data['Label'].unique()

# Extremely Weak : 심한 저체중
# Weak : 저체중
# Normal : 정상체중
# Overweight : 과체중
# Obesity : 비만
# Extreme Obesity : 고도비만

array(['Obesity', 'Normal', 'Overweight', 'Extreme Obesity', 'Weak',
       'Extremely Weak'], dtype=object)

In [30]:
# 데이터 분리
# 1) 문제 데이터(X)
X = data[['Height', 'Weight']] # X(문제) 키, 몸무게 --> 2차원
X

# 2) 정답 데이터(y)
y = data['Label'] # y(정답)
y

0              Obesity
1               Normal
2              Obesity
3           Overweight
4           Overweight
            ...       
495    Extreme Obesity
496            Obesity
497    Extreme Obesity
498    Extreme Obesity
499    Extreme Obesity
Name: Label, Length: 500, dtype: object

In [33]:
# 데이터 크기 확인
print('문제 데이터 : ', X.shape)
print('정답 데이터 : ', y.shape)

문제 데이터 :  (500, 2)
정답 데이터 :  (500,)


In [44]:
# 훈련세트(train set) 테스트 세트(test set)로 행분리
# 한정적인 데이터 내에서 학습 + 평가까지 해야되기 때문에
# 훈련(train) : 70% - 학습(훈련)을 위한 용도
# 테스트(test) : 30% - 학습이 잘 됐는지 확인(테스트)하기 위한 용도

# 총 500개 데이터(0 ~ 499)
# 350개(0 ~ 349) : train
# 150개(350 ~ 499) : test

# X(문제 데이터) - train(70%), test(30%)
# y(답 데이터) - train(70%), test(30%)

# iloc 행분리
X_train = X.iloc[0 : 350 , : ]
X_test = X.iloc[350 : , : ]


y_train = y.iloc[0 : 350]
y_test = y.iloc[350 : ]

# print(X_train)
# print(X_test)
print(y_train)
print(y_test)

0              Obesity
1               Normal
2              Obesity
3           Overweight
4           Overweight
            ...       
345            Obesity
346             Normal
347             Normal
348    Extreme Obesity
349             Normal
Name: Label, Length: 350, dtype: object
350             Normal
351             Normal
352             Normal
353    Extreme Obesity
354     Extremely Weak
            ...       
495    Extreme Obesity
496            Obesity
497    Extreme Obesity
498    Extreme Obesity
499    Extreme Obesity
Name: Label, Length: 150, dtype: object
